In [1]:
!nvidia-smi

Tue Sep 24 13:13:28 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

**Introduções a Operações Customizadas**

In [16]:
%%writefile exercicio1.cu
#include <iostream>                 // Biblioteca padrão para entrada e saída de dados
#include <fstream>                  // Biblioteca para manipulação de arquivos
#include <string>                   // Biblioteca para manipulação de strings
#include <sstream>                  // Biblioteca para manipulação de fluxos de string
#include <chrono>                   // Biblioteca para medir o tempo de execução do código
using namespace std;
#include <thrust/host_vector.h>     // Inclui a biblioteca Thrust para vetores na CPU



struct saxpy
{
    int a;
    saxpy(int a_) : a(a_) {};  // Construtor que inicializa a constante 'a'

    __host__ __device__
    double operator()(const int& x, const int& y) const {
        return a * x + y;  // Operação que será aplicada a cada elemento de x e y
    }
};


int main(){

  thrust::host_vector<int> d_a(4);
  thrust::host_vector<int> d_b(4);
  thrust::host_vector<int> d_c(4);

  d_a[0] = 1; d_a[1] = 2; d_a[2] = 3; d_a[3] = 4;
  d_b[0] = 5; d_b[1] = 6; d_b[2] = 7; d_b[3] = 8;

  int m = 4;


  thrust::transform(d_a.begin(), d_a.end(), d_b.begin(), d_c.begin(), saxpy(m));

  for (int i = 0; i < d_c.size(); i++) {
        std::cout << "d_c1[" << i << "] = " << d_c[i] << std::endl;
    }

  m = 8;

  thrust::transform(d_a.begin(), d_a.end(), d_b.begin(), d_c.begin(), saxpy(m));

  for (int i = 0; i < d_c.size(); i++) {
        std::cout << "d_c2[" << i << "] = " << d_c[i] << std::endl;
    }

  return 0;

}

Overwriting exercicio1.cu


In [17]:
!nvcc -arch=sm_70 -std=c++14 exercicio1.cu -o exercicio1

In [18]:
!./exercicio1

d_c1[0] = 9
d_c1[1] = 14
d_c1[2] = 19
d_c1[3] = 24
d_c2[0] = 13
d_c2[1] = 22
d_c2[2] = 31
d_c2[3] = 40


**Transformações Unárias e Binárias**

In [19]:
%%writefile exercicio2.cu
#include <iostream>                 // Biblioteca padrão para entrada e saída de dados
#include <fstream>                  // Biblioteca para manipulação de arquivos
#include <string>                   // Biblioteca para manipulação de strings
#include <sstream>                  // Biblioteca para manipulação de fluxos de string
#include <chrono>                   // Biblioteca para medir o tempo de execução do código
using namespace std;
#include <thrust/host_vector.h>     // Inclui a biblioteca Thrust para vetores na CPU


struct square
{
    __host__ __device__
    float operator()(const float& x) const {
        return x * x;  // Elevar ao quadrado
    }
};


int main(){

  thrust::host_vector<int> d_v(4);


  d_v[0] = 1; d_v[1] = 2; d_v[2] = 3; d_v[3] = 4;

  thrust::transform(d_v.begin(), d_v.end(), d_v.begin(), square());

  for (int i = 0; i < d_v.size(); i++) {
        std::cout << "d_v[" << i << "] = " << d_v[i] << std::endl;
    }

  return 0;

}

Writing exercicio2.cu


In [20]:
!nvcc -arch=sm_70 -std=c++14 exercicio2.cu -o exercicio2

In [21]:
!./exercicio2

d_v[0] = 1
d_v[1] = 4
d_v[2] = 9
d_v[3] = 16


**Calculando a Magnitude de um Vetor**

Em GPU

In [30]:
%%writefile exercicio3.cu
#include <iostream>                 // Biblioteca padrão para entrada e saída de dados
#include <fstream>                  // Biblioteca para manipulação de arquivos
#include <string>                   // Biblioteca para manipulação de strings
#include <sstream>                  // Biblioteca para manipulação de fluxos de string
#include <chrono>                   // Biblioteca para medir o tempo de execução do código
using namespace std;
#include <thrust/device_vector.h>     // Inclui a biblioteca Thrust para vetores na CPU
#include <thrust/transform_reduce.h>

struct square
{
    __host__ __device__
    float operator()(const float& x) const {
        return x * x;  // Elevar ao quadrado
    }
};

float magnitude(thrust::device_vector<float>& v) {
    float sum_of_squares = thrust::transform_reduce(v.begin(), v.end(), square(), 0.0f, thrust::plus<float>());
    return std::sqrt(sum_of_squares);
}


int main(){

  thrust::device_vector<float> v(4);

  v[0] = 1.234; v[1] = 2.456; v[2] = 3.678; v[3] = 4.901;

  float mag = magnitude(v);

  // Mostra a magnitude
  std::cout << "Magnitude: " << mag << std::endl;


  return 0;

}

Overwriting exercicio3.cu


In [31]:
!nvcc -arch=sm_70 -std=c++14 exercicio3.cu -o exercicio3

In [32]:
!./exercicio3

Magnitude: 6.71582


Em CPU

In [34]:
%%writefile exercicio3_cpu.cpp
#include <iostream>                 // Biblioteca padrão para entrada e saída de dados
#include <vector>                   // Biblioteca padrão para vetores
#include <numeric>                  // Para std::accumulate
#include <cmath>                    // Para std::sqrt

struct square
{
    float operator()(const float& x) const {
        return x * x;
    }
};

float magnitude(const std::vector<float>& v) {
    float sum_of_squares = std::accumulate(v.begin(), v.end(), 0.0f, [](float sum, float x) {
        return sum + x * x;
    });
    return std::sqrt(sum_of_squares);
}

int main() {
    std::vector<float> v(4);

    v[0] = 1.234; v[1] = 2.456; v[2] = 3.678; v[3] = 4.901;

    float mag = magnitude(v);

    std::cout << "Magnitude: " << mag << std::endl;

    return 0;
}


Writing exercicio3_cpu.cpp


In [35]:
!g++ -std=c++14 exercicio3_cpu.cpp -o exercicio3_cpu

In [36]:
!./exercicio3_cpu

Magnitude: 6.71582


**Otimização com Fusion Kernel**

In [49]:
%%writefile exercicio4.cu
#include <iostream>                 // Biblioteca padrão para entrada e saída de dados
#include <fstream>                  // Biblioteca para manipulação de arquivos
#include <string>                   // Biblioteca para manipulação de strings
#include <sstream>                  // Biblioteca para manipulação de fluxos de string
#include <chrono>                   // Biblioteca para medir o tempo de execução do código
using namespace std;
#include <thrust/device_vector.h>     // Inclui a biblioteca Thrust para vetores na CPU
#include <thrust/transform_reduce.h>

struct variance_op
{
    float mean;
    variance_op(float mean_) : mean(mean_) {}  // Construtor que armazena a média

    __host__ __device__
    float operator()(const float& x) const {
        float diff = x - mean;  // Diferença em relação à média
        return diff * diff;  // Elevar ao quadrado a diferença
    }
};

float calculate_variance(const thrust::device_vector<float>& d_vec, float mean) {
    return thrust::transform_reduce(d_vec.begin(), d_vec.end(), variance_op(mean), 0.0f, thrust::plus<float>()) / d_vec.size();
}


// para facilitar o cálculo da média
float calculate_mean(const thrust::device_vector<float>& d_vec) {
    float sum = thrust::reduce(d_vec.begin(), d_vec.end(), 0.0f, thrust::plus<float>());
    return sum / d_vec.size();
}

int main(){

  thrust::device_vector<float> d_vec(4);

  d_vec[0] = 1.234; d_vec[1] = 2.456; d_vec[2] = 3.678; d_vec[3] = 4.901;

  float mean = calculate_mean(d_vec);

  float variance = calculate_variance(d_vec, mean);

  std::cout << "Média: " << mean << std::endl;
  std::cout << "Variância: " << variance << std::endl;

  thrust::device_vector<float> d_vec2(6);

  d_vec2[0] = 1.234; d_vec2[1] = 2.456; d_vec2[2] = 3.678; d_vec2[3] = 4.901; d_vec2[4] = 5.234; d_vec2[5] = 6.567;

  float mean2 = calculate_mean(d_vec2);

  float variance2 = calculate_variance(d_vec2, mean2);

  std::cout << "Média: " << mean2 << std::endl;
  std::cout << "Variância: " << variance2 << std::endl;


  return 0;

}

Overwriting exercicio4.cu


In [50]:
!nvcc -arch=sm_70 -std=c++14 exercicio4.cu -o exercicio4

In [51]:
!./exercicio4

Média: 3.06725
Variância: 1.86752
Média: 4.01167
Variância: 3.17693
